In [ ]:
import os, sys
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
import pygsheets as pg

In [ ]:
start_date = pd.to_datetime('2025-05-01')
yesterday_date = pd.Timestamp.today().date() - timedelta(days=1)

dfDriverHistories = adbFetch(f"""select driverId, date, status dayEndStatus, operatorId, zoneId, vehicleType, clientId, liveDate, cashWallet/100 cashWallet, pointsWallet/100 pointsWallet, penaltyWallet/100 penaltyWallet, isDefaulter, nonOpsDays from dailyDriversHistories force index(dailyDriversHistories_date_driverId_unique) where date between '{start_date}' and '{yesterday_date}' and isBaasDriver != 1 and status not in ('left', 'deleted', 'terminated') and driverId like 'D%' """)

dfDriverHistories

In [ ]:
dfDriverHistories.nunique()

In [ ]:
dfTxns = prodFetch(f""" select sum(upiCollection) upiCollection, date txnDate, driverId txnDriverId, sum(penalty) penalty, sum(netGMV)/100 netGMV, sum(walletCashUsed)/100 walletCashUsed, sum(pointsUsed)/100 pointUsed, count(*) numberOfSwaps, SUM(CASE WHEN qrType = 'smartCard' THEN 1 ELSE 0 END) AS smartCardTransactionCount, SUM(CASE WHEN qrType = 'vpa' THEN 1 ELSE 0 END) AS vpaTransactionCount from transactions force index(transaction_date_index) where date between '{start_date}' and '{yesterday_date}' and deletedAt is null group by txnDriverId, txnDate """)

dfTxns

In [ ]:
dfTxns['txnDate'] = pd.to_datetime(dfTxns['txnDate']).dt.date

dfTxnsToday = dfTxns[dfTxns['txnDate'] == (yesterday_date)]

dfTxnsToday

In [ ]:
dfDailyDriverHistories = dfDriverHistories[dfDriverHistories['date'] == (yesterday_date)]

dfDailyDriverHistories

In [ ]:
dailyDf = dfDailyDriverHistories.merge(dfTxnsToday, left_on='driverId', right_on='txnDriverId', how='left', indicator=True).drop(columns = ['txnDriverId', 'txnDate'])

dailyDf['status'] = dailyDf.apply(lambda row: 'active' if row['dayEndStatus'] != 'active' and row['_merge'] == 'both' else row['dayEndStatus'], axis=1)

dailyDf.drop(columns = ['_merge', 'dayEndStatus'], inplace=True)

dailyDf.replace(np.nan, 0, inplace=True)

dailyDf

In [ ]:
overallDf = dfDriverHistories.merge(dfTxns, left_on=['driverId', 'date'], right_on=['txnDriverId', 'txnDate'], how='left', indicator=True).drop(columns=['txnDriverId', 'txnDate'])

overallDf['status'] = overallDf.apply(lambda row: 'active' if row['dayEndStatus'] != 'active' and row['_merge'] == 'both' else row['dayEndStatus'], axis=1)

overallDf.drop(columns = ['_merge', 'dayEndStatus'], inplace=True)

overallDf.replace(np.nan, 0, inplace=True)

overallDf

In [ ]:
dfFeatureWallet = prodFetch(f"""select date, cashAmount/100 cashAmount, pointAmount/100 pointsamount, driverId, featureOrderId from featureDriverWallets where date between '{start_date}' and '{yesterday_date}' and deletedAt is null and paymentFor in ('walletRecharge', 'driverKhataRecharge') """)

dfFeatureWallet

In [ ]:
dfFeatureOrders = prodFetch(f"""select id, orderCreatedFor, orderCreatedBy, DATE(DATE_ADD(createdAt, INTERVAL 330 MINUTE)) orderDate from featureOrders force index(orderCreatedFor) where deletedAt is null and DATE(DATE_ADD(createdAt, INTERVAL 330 MINUTE)) between '{start_date}' and '{yesterday_date}' and orderCreatedFor is not null and orderCreatedFor like 'D%'""")  

dfFeatureOrders

In [ ]:
df_result = dfFeatureOrders.merge(dfFeatureWallet, left_on=['id', 'orderDate'], right_on=['featureOrderId', 'date'], how='inner')

df_result = df_result[dfFeatureWallet.columns]

df_result.drop(columns=['featureOrderId'], inplace=True)

df_result['WalletRechargeCount'] = 1

df_result.groupby(['driverId', 'date']).agg({'cashAmount': 'sum', 'pointsamount': 'sum', 'WalletRechargeCount': 'sum'}).reset_index()

df_result

In [ ]:
df_result.groupby(['driverId', 'date']).agg({'cashAmount': 'sum', 'pointsamount': 'sum', 'WalletRechargeCount': 'sum'}).reset_index().rename(columns={'cashAmount': 'walletRechargeCashAmount', 'pointsamount': 'walletRechargePointsAmount', 'WalletRechargeCount': 'walletRechargeCount'})

df_result

In [ ]:
df_result['date'] = pd.to_datetime(df_result['date']).dt.date

df_resultDaily = df_result[df_result['date'] == yesterday_date]

df_resultDaily

In [ ]:
# dailyDf = dailyDf.merge(df_resultDaily, on=['driverId', 'date'], how='left')

# dailyDf

In [ ]:
# overallDf = overallDf.merge(df_result, on=['driverId', 'date'], how='left')

# overallDf

In [ ]:
dfCards = prodFetch(f"""select occupant, status smartCardStatus, createdAt, id smartCardId from smartCards where deletedAt is null and status = 'active' """)

dfCards

In [ ]:
dfCards.nunique()

In [ ]:
df_latest_id = dfCards.loc[dfCards.groupby('occupant')['createdAt'].idxmax(), ['occupant', 'smartCardId']]

df_latest_id

In [ ]:
dailyDf = dailyDf.merge(df_latest_id, left_on='driverId', right_on='occupant', how='left').drop(columns=['occupant'])

dailyDf

In [ ]:
dailyDf['smartCardOccupancyFlag'] = dailyDf.apply(lambda x: 1 if pd.notna(x['smartCardId']) else 0, axis=1)

dailyDf

In [ ]:
dfCardsStatusLogs = prodFetch(f"""
    SELECT smartCardId, status, driverId, date
    FROM (
        SELECT 
            smartCardId,
            status,
            occupant AS driverId,
            date(DATE_ADD(createdAt, INTERVAL 330 MINUTE)) AS date,
            ROW_NUMBER() OVER (PARTITION BY smartCardId ORDER BY date(DATE_ADD(createdAt, INTERVAL 330 MINUTE)) DESC) AS rn
        FROM smartCardStatusLogs
        WHERE date(DATE_ADD(createdAt, INTERVAL 330 MINUTE)) 
            BETWEEN '{start_date}' AND '{yesterday_date}'
    ) AS sub
    WHERE rn = 1
""")

dfCardsStatusLogs

In [ ]:
overallDf = overallDf.merge(df_latest_id, left_on='driverId', right_on='occupant', how='left').drop(columns=['occupant'])

overallDf

In [ ]:
overallDf['smartCardOccupancyFlag'] = overallDf.apply(lambda x: 1 if pd.notna(x['smartCardId']) else 0, axis=1)

overallDf

In [ ]:
dfLoyalPartner = prodFetch(f"""
WITH filteredTransactions AS (
    SELECT 
        driverId, 
        partnerId, 
        COUNT(*) AS txns, 
        MAX(createdAt) AS lastTxnTime
    FROM transactions
    WHERE 
        createdAt >= DATE_SUB(CURDATE(), INTERVAL 28 DAY)
        AND driverId LIKE 'D%'
        AND deletedAt IS NULL
    GROUP BY driverId, partnerId
),
rankedPartners AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (
            PARTITION BY driverId 
            ORDER BY txns DESC, lastTxnTime DESC
        ) AS rn
    FROM filteredTransactions
)
SELECT 
    driverId, 
    partnerId AS StickyPartnerId
FROM rankedPartners
WHERE rn = 1
""")

dfLoyalPartner

In [ ]:
# dfLoyalPartner = prodFetch(f"""
# WITH FilteredTransactions AS (
#     SELECT
#         driverId,
#         partnerId,
#         COUNT(*) AS Transactions
#     FROM
#         transactions
#     WHERE
#         createdAt >= DATE_SUB(CURDATE(), INTERVAL 90 DAY)
#         AND driverId like 'D%'
#         AND deletedAt IS NULL
#     GROUP BY
#         driverId, partnerId
# ),
# DriverTotals AS (
#     SELECT
#         driverId,
#         SUM(Transactions) AS TotalTransactions
#     FROM
#         FilteredTransactions
#     GROUP BY
#         driverId
# ),
# StickyPartner AS (
#     SELECT
#         ft.driverId,
#         ft.partnerId AS StickyPartnerId,
#         ft.Transactions AS StickyPartnerTransactions
#     FROM
#         FilteredTransactions ft
#     JOIN (
#         SELECT
#             driverId,
#             MAX(Transactions) AS MaxTransactions
#         FROM
#             FilteredTransactions
#         GROUP BY
#             driverId
#     ) MaxTransactionsTable
#     ON
#         ft.driverId = MaxTransactionsTable.driverId
#         AND ft.Transactions = MaxTransactionsTable.MaxTransactions
# )
# SELECT
#     sp.driverId,
#     sp.StickyPartnerId
# FROM
#     StickyPartner sp
# JOIN
#     DriverTotals dt
# ON
#     sp.driverId = dt.driverId
# WHERE
#     (sp.StickyPartnerTransactions / dt.TotalTransactions) >= 0.75
# """)

# dfLoyalPartner

In [ ]:
# dfAllPartnersWithPercent = prodFetch(f"""
# WITH FilteredTransactions AS (
#     SELECT
#         driverId,
#         partnerId,
#         COUNT(*) AS Transactions
#     FROM
#         transactions
#     WHERE
#         createdAt >= DATE_SUB(CURDATE(), INTERVAL 90 DAY)
#         AND driverId like 'D%'
#         AND deletedAt IS NULL
#     GROUP BY
#         driverId, partnerId
# ),
# DriverTotals AS (
#     SELECT
#         driverId,
#         SUM(Transactions) AS TotalTransactions
#     FROM
#         FilteredTransactions
#     GROUP BY
#         driverId
# )
# SELECT
#     ft.driverId,
#     ft.partnerId,
#     ft.Transactions,
#     dt.TotalTransactions,
#     ROUND((ft.Transactions / dt.TotalTransactions) * 100, 2) AS PercentageOfTransactions
# FROM
#     FilteredTransactions ft
# JOIN
#     DriverTotals dt
# ON
#     ft.driverId = dt.driverId
# ORDER BY
#     ft.driverId, PercentageOfTransactions DESC
# """)

# dfAllPartnersWithPercent

In [ ]:
dfLoyalPartner.nunique()

In [ ]:
# dfLoyalPartnerGrouped = dfLoyalPartner.groupby('driverId')['StickyPartnerId'].apply(lambda x: ', '.join(x)).reset_index()

# dfLoyalPartnerGrouped

In [ ]:
dailyDf = dailyDf.merge(dfLoyalPartner, on='driverId', how='left')

dailyDf

In [ ]:
dfLoyalPartnerPast = prodFetch(f"""
WITH RECURSIVE dateSeries AS (
    SELECT DATE('{start_date}') AS date
    UNION ALL
    SELECT DATE_ADD(date, INTERVAL 1 DAY)
    FROM dateSeries
    WHERE date <= '{yesterday_date}'
),
filteredTransactions AS (
    SELECT 
        ds.date,
        t.driverId, 
        t.partnerId, 
        COUNT(*) AS txns, 
        MAX(t.createdAt) AS lastTxnTime
    FROM dateSeries ds
    JOIN transactions t 
        ON t.createdAt >= DATE_SUB(ds.date, INTERVAL 28 DAY)
        AND t.createdAt < ds.date
        AND t.driverId LIKE 'D%'
        AND t.deletedAt IS NULL
    GROUP BY ds.date, t.driverId, t.partnerId
),
rankedPartners AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (
            PARTITION BY date, driverId 
            ORDER BY txns DESC, lastTxnTime DESC
        ) AS rn
    FROM filteredTransactions
)
SELECT 
    date,
    driverId, 
    partnerId AS StickyPartnerId
FROM rankedPartners
WHERE rn = 1
""")

dfLoyalPartnerPast

In [ ]:
dfLoyalPartnerPast.sort_values(by=['driverId', 'date'], ascending=[True, True], inplace=True)
dfLoyalPartnerPast['date'] = pd.to_datetime(dfLoyalPartnerPast['date']).dt.date

dfLoyalPartnerPast.reset_index(drop=True, inplace=True)
dfLoyalPartnerPast

In [ ]:
overallDf = overallDf.merge(dfLoyalPartnerPast, on=['driverId', 'date'], how='left')

overallDf

In [ ]:
dailyDf.replace(np.nan, "", inplace=True)

dailyDf

In [ ]:
overallDf.replace(np.nan, "", inplace=True)

overallDf

# -------------------------

In [ ]:
conn = psycopg2.connect(dbname = "operations_manager_prod", user = "sarthak_sachdev", password = "Sarthak@12345", host = "operation.replica.upgrid.in", port = "5432")

print('PostgreSQL Connection Established')

# Fetch Data from PostgreSQL server
query1 = f'''select complainant_id, (created_at + INTERVAL '330 minutes')::DATE as ticket_date, count(*) totalTickets from tickets where (created_at + INTERVAL '330 minutes')::DATE between '{start_date}' and '{yesterday_date}' and deleted_at is null and complainant_id like 'D%' group by complainant_id, ticket_date
'''

dfTickets = pd.read_sql(query1, conn)

conn.close()

dfTickets

In [ ]:
dfTickets.nunique()

In [ ]:
dfTicketsDaily = dfTickets[dfTickets['ticket_date'] == (yesterday_date)]

dfTicketsDaily

In [ ]:
dailyDf = dailyDf.merge(dfTicketsDaily, left_on=['driverId', 'date'], right_on=['complainant_id', 'ticket_date'], how='left').drop(columns=['complainant_id', 'ticket_date'])

dailyDf

In [ ]:
overallDf = overallDf.merge(dfTickets, left_on=['driverId', 'date'], right_on=['complainant_id', 'ticket_date'], how='left').drop(columns=['complainant_id', 'ticket_date'])

overallDf

In [ ]:
dailyDf.replace(np.nan, 0, inplace=True)

dailyDf

In [ ]:
overallDf.replace(np.nan, 0, inplace=True)

overallDf

# -------------------------------

In [ ]:
dfUsers = prodFetch("""select employeeId, mobile, alternateMobile from users where employeeId like 'D%' group by 1 """)

dfUsers.replace(np.nan, "", inplace=True)

dfUsers

In [ ]:
dfUsers.nunique()

In [ ]:
dfCallingNumber = prodFetch(""" select customerId, callingNumber, DATE_ADD(createdAt, INTERVAL 330 MINUTE) AS callDate, count, createdAt from customerCallingNumbers where deletedAt is null""")

dfCallingNumber

In [ ]:
dfCallingNumber['date'] = pd.to_datetime(dfCallingNumber['callDate'])
dfCallingNumber['createdAt'] = pd.to_datetime(dfCallingNumber['createdAt'])

dfCallingNumber_sorted = dfCallingNumber.sort_values(by=['count', 'createdAt'], ascending=[False, False])

df_filtered = dfCallingNumber_sorted.drop_duplicates(subset=['customerId'], keep='first')

dfNumbers = dfUsers.merge(df_filtered, left_on='employeeId', right_on='customerId', how='left').drop(
    columns=['customerId', 'count', 'callDate', 'date', 'createdAt']
)

dfNumbers

In [ ]:
dfNumbers.nunique()

In [ ]:
dfNumbers['callingNumber'] = dfNumbers['callingNumber'].astype(str).str.replace('NaN', '')

sample = dfNumbers[dfNumbers['callingNumber'] != "nan"].copy()

sample

In [ ]:
dfNumbers.replace(np.nan, "", inplace=True)

dfNumbers

In [ ]:
dfNumbers.nunique()

In [ ]:
dailyDf = dailyDf.merge(dfNumbers, left_on='driverId', right_on='employeeId', how='left').drop(columns=['employeeId'])

dailyDf

In [ ]:
overallDf = overallDf.merge(dfNumbers, left_on='driverId', right_on='employeeId', how='left').drop(columns=['employeeId'])

overallDf

In [ ]:
# dfLeaves = prodFetch(f"""SELECT driverId, date, SUM(1) OVER (PARTITION BY driverId ORDER BY date) as nonOpsDays FROM driverLeaves WHERE deletedAt IS NULL AND driverId LIKE 'D%' AND date >= '20250301' ORDER BY driverId, date""")

# dfLeaves

In [ ]:
# dfTxn = prodFetch(f"""WITH UniqueTxns AS (SELECT DISTINCT driverId, date FROM transactions WHERE deletedAt IS NULL AND driverId LIKE 'D%' AND date >= '20250301') SELECT driverId, date, SUM(1) OVER (PARTITION BY driverId ORDER BY date) as OpsDays FROM UniqueTxns WHERE date >= '20250301' ORDER BY driverId, date""")

# dfTxn

In [ ]:
# dfdates = adbFetch(f""" select driverId, date from dailyDriversHistories where date between '20250301' and '20250331' and isBaasDriver != 1 and status not in ('left', 'deleted', 'terminated') and driverId like 'D%' """)

# dfdates

In [ ]:
# dfLeaves['date'] = pd.to_datetime(dfLeaves['date'])
# dfTxn['date'] = pd.to_datetime(dfTxn['date'])
# dfdates['date'] = pd.to_datetime(dfdates['date'])

# dfLeaves_full = dfdates.merge(dfLeaves, on=['date', 'driverId'], how='left')
# dfTxn_full = dfdates.merge(dfTxn, on=['date', 'driverId'], how='left')

# dfLeaves_full['nonOpsDays'] = dfLeaves_full.groupby('driverId')['nonOpsDays'].ffill().fillna(0)
# dfTxn_full['OpsDays'] = dfTxn_full.groupby('driverId')['OpsDays'].ffill().fillna(0)

# dfLeaves_full = dfLeaves_full.reset_index(drop=True)
# dfTxn_full = dfTxn_full.reset_index(drop=True)

# dfLeaves_full['driverId'] = dfLeaves_full['driverId'].ffill()
# dfTxn_full['driverId'] = dfTxn_full['driverId'].ffill()

# print(dfLeaves_full)
# print(dfTxn_full)

In [ ]:
# dfDsh = prodFetch(f"""select driverId, date, SUM(1) OVER (PARTITION BY driverId ORDER BY date) as activeDays from driverStatusHistories where driverId like 'D%' and date between '20250301' and '20250331' and deletedAt is null and status in ('active')""")

# dfDsh

In [ ]:
# dfDsh['date'] = pd.to_datetime(dfDsh['date'])

# dfDsh_full = dfdates.merge(dfDsh, on=['date', 'driverId'], how='left')

# dfDsh_full['driverId'] = dfDsh_full['driverId'].ffill()
# dfDsh_full['activeDays'] = dfDsh_full.groupby('driverId')['activeDays'].ffill().fillna(0)

# dfDsh_full

In [ ]:
# dfDsh_full

In [ ]:
# dfDays = dfTxn_full.merge(dfLeaves_full, on=['date', 'driverId'], how='left').merge(dfDsh_full, on=['date', 'driverId'], how='left')
# dfDays['date'] = pd.to_datetime(dfDays['date']).dt.date

# dfDays

In [ ]:
# dailyDf

In [ ]:
# overallDf

In [ ]:
# daily = dailyDf.merge(dfDays, on=['date', 'driverId'], how='left')

# daily

In [ ]:
# overall = overallDf.merge(dfDays, on=['date', 'driverId'], how='left')

# overall

In [ ]:
dfLeaves = prodFetch(f"""select driverId, date, 1 as nonOpsFlag FROM driverLeaves force index(driverId) WHERE deletedAt IS NULL AND driverId LIKE 'D%' AND date between '{start_date}' and '{yesterday_date}'""")

dfLeaves

In [ ]:
dfTxn = prodFetch(f"""SELECT distinct driverId, date from transactions force index(transaction_date_index) WHERE date between '{start_date}' and '{yesterday_date}' and deletedAt IS NULL and driverId LIKE 'D%' """)

dfTxn['opsFlag'] = 1

dfTxn

In [ ]:
dailyDf = dailyDf.merge(dfTxn, on=['driverId', 'date'], how='left').merge(dfLeaves, on=['driverId', 'date'], how='left')

dailyDf.fillna(0, inplace=True)

dailyDf

In [ ]:
overallDf = overallDf.merge(dfTxn, on=['driverId', 'date'], how='left').merge(dfLeaves, on=['driverId', 'date'], how='left')
overallDf.fillna(0, inplace=True)
overallDf

In [ ]:
date_range = pd.date_range(start=start_date, end=yesterday_date)

buffer_days = 90
min_date = start_date - timedelta(days=buffer_days)

In [ ]:
dfDsh = prodFetch(f"""select driverId, date from driverStatusHistories where driverId like 'D%' and date >= '{min_date}' and deletedAt is null and status in ('active')""")

dfDsh

In [ ]:
dfTrxn = prodFetch(f"""select distinct driverId, date from transactions where driverId like 'D%' and date >= '{min_date}' and deletedAt is null""")

dfTrxn

In [ ]:
result = []

dfDsh['date'] = pd.to_datetime(dfDsh['date'])
dfTrxn['date'] = pd.to_datetime(dfTrxn['date'])

for current_date in date_range:
    window_start = current_date - timedelta(days=90)
    
    active = dfDsh[(dfDsh['date'] > window_start) & (dfDsh['date'] <= current_date)]
    active_count = active.groupby('driverId').size().reset_index(name='activeDays')

    ops = dfTrxn[(dfTrxn['date'] > window_start) & (dfTrxn['date'] <= current_date)]
    ops_count = ops.groupby('driverId').size().reset_index(name='opsDaysCount')

    merge = active_count.merge(ops_count, on='driverId', how='outer').fillna(0)
    merge['date'] = current_date
    merge['ops/Active'] = merge['opsDaysCount'] / merge['activeDays']
    merge['ops/Active'] = merge['ops/Active'].replace([np.inf, -np.inf, np.nan], 0)

    result.append(merge)

final_df = pd.concat(result).reset_index(drop=True)

final_df

In [ ]:
dailyDf['date'] = pd.to_datetime(dailyDf['date'])
final_df['date'] = pd.to_datetime(final_df['date'])

dailyDf = dailyDf.merge(final_df, on=['driverId', 'date'], how='left')
dailyDf.fillna(0, inplace=True)

dailyDf

In [ ]:
overallDf['date'] = pd.to_datetime(overallDf['date'])
final_df['date'] = pd.to_datetime(final_df['date'])

overallDf = overallDf.merge(final_df, on=['driverId', 'date'], how='left')
overallDf.fillna(0, inplace=True)

overallDf

In [ ]:
print(dailyDf.columns)
print(overallDf.columns)

In [ ]:
int_cols = [
    'isDefaulter', 'nonOpsDays', 'numberOfSwaps',
    'smartCardTransactionCount', 'vpaTransactionCount',
    'smartCardOccupancyFlag', 'totaltickets',
    'opsFlag', 'nonOpsFlag'
]

float_cols = [
    'cashWallet', 'pointsWallet', 'penaltyWallet',
    'upiCollection', 'penalty', 'netGMV',
    'walletCashUsed', 'pointUsed', 'ops/Active'
]

date_cols = ['date', 'liveDate']

all_cols = [
    'driverId', 'date', 'operatorId', 'zoneId', 'vehicleType', 'clientId',
    'liveDate', 'cashWallet', 'pointsWallet', 'penaltyWallet',
    'isDefaulter', 'nonOpsDays', 'upiCollection', 'penalty', 'netGMV',
    'walletCashUsed', 'pointUsed', 'numberOfSwaps',
    'smartCardTransactionCount', 'vpaTransactionCount', 'status',
    'smartCardId', 'smartCardOccupancyFlag', 'StickyPartnerId',
    'totaltickets', 'mobile', 'alternateMobile', 'callingNumber', 'opsFlag',
    'nonOpsFlag', 'ops/Active'
]

string_cols = list(set(all_cols) - set(int_cols) - set(float_cols) - set(date_cols))

for col in int_cols:
    dailyDf[col] = dailyDf[col].fillna(0).astype(int)
    overallDf[col] = overallDf[col].fillna(0).astype(int)

for col in float_cols:
    dailyDf[col] = dailyDf[col].astype(float)
    overallDf[col] = overallDf[col].astype(float)

for col in date_cols:
    dailyDf[col] = pd.to_datetime(dailyDf[col])
    overallDf[col] = pd.to_datetime(overallDf[col])

for col in string_cols:
    dailyDf[col] = dailyDf[col].astype(str)
    overallDf[col] = overallDf[col].astype(str)

In [ ]:
dailyDf.info()

In [ ]:
overallDf.info()